## 직방 사이트의 데이터를 이용한, 원하는 지역의 월세 데이터 크롤링

- 참조 => 직방 데이터 크롤링 레퍼런스 [https://www.inflearn.com/questions/22997/%EC%A7%81%EB%B0%A9-%ED%81%AC%EB%A1%A4%EB%A7%81%EC%97%90%EC%84%9C-%EC%A7%88%EB%AC%B8%EC%9E%88%EC%8A%B5%EB%8B%88%EB%8B%A4]


### 크롤링 조건
- 월세 매물
- 서울 지역의 매물
- 모든 데이터를 크롤링 해오진 못한 것으로 예상, 레퍼런스 참조하며 크롤링 진행했지만 나만의 방식으로 진행하진 못했음.

In [1]:
# !pip install geohash2 

In [2]:
import pandas as pd
import requests
import pprint
import geohash2

In [3]:
# 서울 구 리스트 + 광명시 + 부천시 => 전 분석 보고서에서 필요한 키워드임을 정리
sgg_list = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구','은평구','서대문구'\
            ,'마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구', '광명시',\
           '부천시']

#### 빈 데이터 프레임 만들기

In [4]:
df = pd.DataFrame(columns=['보증금','월세','서비스형태','매물형태','지역명'])

In [5]:
df

,보증금,월세,서비스형태,매물형태,지역명


### 크롤링 코드 정리

In [6]:
# 검색 키워드
keyword = "부천시"

url = "https://apis.zigbang.com/search?q={}".format(keyword)

req = requests.get(url)
# 실제 api 주소에서 json 형태로 리턴되기 때문에 json 형태로 값을 받습니다.
_json = req.json()

if _json.get("code") == "200":
    # 검색어에 해당하는 자동완성값은 여러개인데 그중에 맨 위에 [0] 번째 한가지에 대해서만 검색.
    # 예를 들면 부천시 라고 검색창에 입력하면 아래에 부천시, 부천시 상동 등등 예시로 나오는 검색리스트가 있음 
    # 그중에서 첫번째를 선택한다는 의미
    data = _json.get("items")[0]
    _description = data.get("description")
    _id = data.get("id")
    _lat = data.get("lat")
    _lng = data.get("lng")
    _zoom = data.get("zoom")

    geohash = geohash2.encode(_lat, _lng, precision=5)

    # 위에서 구한 geohash 값을 아래의 api 로 호출하고 쿼리(전세 월세 등)를 넘겨주는 주소.
    url = "https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=월세&service_type_eq=원룸".format(geohash)

    _req_items = requests.get(url).json()

    # items 값은 실제 매물 데이터의 인덱스 값.
    _items = _req_items.get("items")

    item_ids = []
    for item in _items:
        item_ids.append(item.get("item_id"))
        
    items = {"item_ids": item_ids[:10000]}

    _results = requests.post('https://apis.zigbang.com/v2/items/list', data=items).json()
    datas = _results.get("items")
    
    # 매물 목록에 대해 for문 적용해서 필요한 요소 추출
    for d in datas:
        _address = "{} {}".format(d.get("address1"), d.get("address2"))
        if d.get("address3") is not None:
            _address += " {}".format(d.get("address3"))

        building_floor = d.get("building_floor")
        floor = d.get("floor")
        thumbnail = d.get("images_thumbnail")
        item_id = d.get("item_id")
        reg_date = d.get("reg_date")
        sales_type = d.get("sales_type")
        service_type = d.get("service_type")
        size_m2 = d.get("size_m2")
        title = d.get("title")
        deposit = d.get("deposit")
        rent = d.get("rent")
        local = d.get("addressOrigin")['local2']
        
        print("*" * 100)
        print("{} [{}]".format(title, item_id))
        print("보증금/월세: {}/{}".format(deposit, rent))
        print("건물층/매물층: {}/{}".format(building_floor, floor))
        print("등록일자: {}".format(reg_date))
        print("서비스형태/매물형태: {}/{}".format(service_type, sales_type))
        print("사이즈: {}".format(size_m2))
        print(local)


****************************************************************************************************
v.부천역 공실 보증금조절가능 [37554167]
보증금/월세: 200/33
건물층/매물층: 5/4
등록일자: 2023-07-29T17:10:38+09:00
서비스형태/매물형태: 원룸/월세
사이즈: 16.53
부천시
****************************************************************************************************
부천역 5분거리 새로 리모델한 베란다 있는 빌라식 주택 [37714772]
보증금/월세: 500/45
건물층/매물층: 4/1
등록일자: 2023-08-13T17:08:26+09:00
서비스형태/매물형태: 원룸/월세
사이즈: 33.0578
부천시
****************************************************************************************************
부개역,송내역 인근, 내부 깨끗하게 수리 된 투룸 [37679979]
보증금/월세: 1000/60
건물층/매물층: 5/5
등록일자: 2023-08-09T23:02:00+09:00
서비스형태/매물형태: 빌라/월세
사이즈: 40.8
부평구
****************************************************************************************************
O N L Y丶부천역5분丶빌트인 굿丶단기 ok [37696335]
보증금/월세: 60/55
건물층/매물층: 9/중
등록일자: 2023-08-11T14:21:21+09:00
서비스형태/매물형태: 오피스텔/월세
사이즈: 39.67
부천시
***************************************************************************

### 함수 만들기

In [7]:
def giggbbang(keyword):
    keyword = keyword
    df = pd.DataFrame(columns=['보증금','월세','서비스형태','매물형태','지역명'])

    url = "https://apis.zigbang.com/search?q={}".format(keyword)

    req = requests.get(url)
    _json = req.json()

    if _json.get("code") == "200":
        data = _json.get("items")[0]
        _description = data.get("description")
        _id = data.get("id")
        _lat = data.get("lat")
        _lng = data.get("lng")
        _zoom = data.get("zoom")

        geohash = geohash2.encode(_lat, _lng, precision=5)

        url = "https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=월세&service_type_eq=원룸".format(geohash)
        _req_items = requests.get(url).json()
        _items = _req_items.get("items")

        item_ids = []
        for item in _items:
            item_ids.append(item.get("item_id"))

        items = {"item_ids": item_ids[:10000]}

        _results = requests.post('https://apis.zigbang.com/v2/items/list', data=items).json()
        datas = _results.get("items")

        if(datas is not None):
            for d in datas:
                _address = "{} {}".format(d.get("address1"), d.get("address2"))
                if d.get("address3") is not None:
                    _address += " {}".format(d.get("address3"))

                building_floor = d.get("building_floor")
                floor = d.get("floor")
                thumbnail = d.get("images_thumbnail")
                item_id = d.get("item_id")
                reg_date = d.get("reg_date")
                sales_type = d.get("sales_type")
                service_type = d.get("service_type")
                size_m2 = d.get("size_m2")
                title = d.get("title")
                deposit = d.get("deposit")
                rent = d.get("rent")
                local = d.get("addressOrigin")['local2']

                # 빈 데이터 프레임에 반복적으로 concat으로 데이터 추가
                temp = pd.DataFrame({'보증금':deposit,'월세':rent,'서비스형태':service_type,'매물형태':sales_type,'지역명':local},index=[0])
                df = pd.concat([df,temp],ignore_index=True) 
    return df

In [8]:
gig_gangnam = giggbbang('강남구')
gig_gangnam

,보증금,월세,서비스형태,매물형태,지역명
0,100,110,원룸,월세,강남구
1,100,120,원룸,월세,강남구
2,100,130,원룸,월세,강남구
3,100,110,원룸,월세,강남구
4,150,150,원룸,월세,강남구
...,...,...,...,...,...
520,5000,430,오피스텔,월세,강남구
521,3000,120,원룸,월세,강남구
522,3000,146,원룸,월세,강남구
523,1000,60,원룸,월세,광진구


In [9]:
total_df = pd.DataFrame(columns=['보증금','월세','서비스형태','매물형태','지역명'])

for i in sgg_list:
    temp = giggbbang(i)
    total_df = pd.concat([total_df, temp])

In [10]:
total_df

,보증금,월세,서비스형태,매물형태,지역명
0,400,30,원룸,월세,용산구
1,1000,75,오피스텔,월세,용산구
2,5000,80,오피스텔,월세,용산구
3,1000,90,오피스텔,월세,용산구
4,2000,25,원룸,월세,용산구
...,...,...,...,...,...
347,300,95,빌라,월세,부천시
348,300,140,빌라,월세,부천시
349,300,135,빌라,월세,부천시
350,300,90,오피스텔,월세,계양구


In [11]:
total_df.drop_duplicates(inplace=True, ignore_index=True)
total_df

,보증금,월세,서비스형태,매물형태,지역명
0,400,30,원룸,월세,용산구
1,1000,75,오피스텔,월세,용산구
2,5000,80,오피스텔,월세,용산구
3,1000,90,오피스텔,월세,용산구
4,2000,25,원룸,월세,용산구
...,...,...,...,...,...
4628,300,95,빌라,월세,부천시
4629,300,140,빌라,월세,부천시
4630,300,135,빌라,월세,부천시
4631,300,90,오피스텔,월세,계양구


In [12]:
total_df.loc[total_df['지역명']=='부천시']

,보증금,월세,서비스형태,매물형태,지역명
4456,200,33,원룸,월세,부천시
4457,500,45,원룸,월세,부천시
4459,60,55,오피스텔,월세,부천시
4460,1000,32,원룸,월세,부천시
4461,55,55,오피스텔,월세,부천시
...,...,...,...,...,...
4627,9000,15,빌라,월세,부천시
4628,300,95,빌라,월세,부천시
4629,300,140,빌라,월세,부천시
4630,300,135,빌라,월세,부천시


## 크롤링한 데이터로 지도 시각화 예정

In [13]:
# total_df.to_csv('./data/직방_크롤링_데이터.csv')